In [1]:
import cv2
import numpy as np
import os

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [2]:
DATA_PATH = 'MP_DATA_final/'

# Classes/Ações que queremos detectar: (Podemos expandir isso!!)
actions = np.array([chr(i) for i in range(ord('A'), ord('I') + 1)])
 
number_of_videos = 60 # Número de sequências ou vídeos que iremos coletar os frames (Pode ser modificado!)
sequence_length = 180 # Para cada vídeo, iremos coletar 30 frames. (Pode ser modificado!)

### Preprocessamento:

In [3]:
# Dictonary comprehension para relacionar o número ao texto da classe:
label_map = {label:num for num, label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in range(number_of_videos):
        janela = []
        for numero_frame in range(sequence_length):
            res = np.load(DATA_PATH +  action + '/' +str(sequence) + '/' + "{}.npy".format(numero_frame))
            janela.append(res)
            
        # Juntando as sequências:
        sequences.append(janela)

        # Juntando o respectivo label da sequência:
        labels.append(label_map[action])

In [4]:
# Transformando as features e o target em arrays:
x = np.array(sequences)
y = to_categorical(labels).astype(int)

In [8]:
# Separando em conjunto de treinamento e teste:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

### Treinamento:

In [ ]:
# Conseguimos ver o andamento do Treinamento:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# Arquitetura do modelo:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu')) # return_sequences False, pois não iremos retornar as sequencias para a camada Fully connected.
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [ ]:
# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
# Treinar o modelo
model.fit(X_train, y_train, epochs=1700, callbacks=[tb_callback])

In [ ]:
# Resumno sobre os paámetros treináveis do modelo:
model.summary()

In [ ]:
# Predições do modelo no conjunto de teste:
resultado = model.predict(X_test)

# Criando uma lista com o index da classe predita e da classe real:
y_pred = [np.argmax(line) for line in resultado]
y_test_pos = [np.argmax(line) for line in y_test]

In [ ]:
# Calculando acurácia no conjunto de teste:
acuracia = accuracy_score(y_test_pos, y_pred)
print(f"Acurácia - Test: {acuracia}")

In [ ]:
# Salvando o modelo
model.save(r'modelos\modelo6.keras')